In [11]:
!pip install nltk
!pip install rouge-score
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 5.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 kB 5.6 MB/s eta 0:00:000:00:01m eta 0:00:01


In [25]:
import os
import csv
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
class EvaluateModel(object):

    categories = ['goal','description','scope','requirements','architecture','users','summary']
    
    
    
    def __init__(self):
        print('Evaluating Results...')
        self.path = os.getcwd()+'/../../Data'
        self.categories = ['goal','description','scope','requirements','architecture','users','summary']
        self.evalData = self.path + "/evaluation_data/evaluation_data.csv"
        self.df = None
        self.readEvaluation()
        self.getBleuScore()
        self.scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
        self.getRougeScore()
        self.writeResults()
    
    def tokenize(self, column):
        column = column.apply(lambda x: x.split())
        return column
    
    def readEvaluation(self):
        self.df = pd.read_csv(self.evalData)
        self.df.fillna("", inplace = True)
    
    def getBleuScore(self):
        df = self.df.copy()
        df.predicted = self.tokenize(df.predicted)
        df.expected = self.tokenize(df.expected)
        scores = []
        for index, row in df.iterrows():
            blue_score = sentence_bleu(list(df.expected), row.predicted)
            scores.append(bleu_score)
        self.df["bleu_score"] = scores
    
    def getRougeScore(self):
        df = self.df.copy()
        rPrec = []
        rRec = []
        rF = []
        
        for index, row in df.iterrows():
            rouge_score = scorer.score(row.expected, row.predicted)
            rPrec.append(rouge_score['rouge1'].precision)
            rRec.append(rouge_score['rouge1'].recall)
            rF.append(rouge_score['rouge1'].fmeasure)
        
        self.df["rouge_precision"] = rPrec
        self.df["rouge_recall"] = rRec
        self.df["rouge_fmeasure"] = rF
    
    def writeResults(self):
        df.to_csv(self.path + "/evaluation_data/evaluated_data.csv")
    
#EvaluateModel()

Evaluating Results...
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['goal']
['description']
['scope']
['requirements']
['architecture']
['users']
['summary']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]